In [1]:
from sys import path
path.append(r"../")

dataDir='../Data/Causal/'
storage_path='../Data/'

In [2]:
from utils import Syn_Generator_OOD

Syn_2442 = Syn_Generator_OOD(n=1000, ate=0,sc=1,sh=0,one=1,depX=0.05,depU=0.05,VX=1,mV=2,mX=4,mU=4,mXs=2,init_seed=7,seed_coef=10,details=1,storage_path=storage_path)
Syn_2442.run(n=1000, num_reps=10)

SynOOD_1_0_1_0.05_0.05_1
##############################
The data path is: ../Data//data/SynOOD_1_0_1_0.05_0.05_1/2_4_4_2/
The ATE:
------------------------------
ate: 1
------------------------------
Next, run dataGenerator: 
Run 0/10. 
Run 1/10. 
Run 2/10. 
Run 3/10. 
Run 4/10. 
Run 5/10. 
Run 6/10. 
Run 7/10. 
Run 8/10. 
Run 9/10. 
------------------------------


In [3]:
Datasets = [Syn_2442]

In [4]:
import os
import argparse
import pandas as pd
import numpy as np
import torch

from utils import log, CausalDataset
from module.Regression_OOD import run as run_Reg

os.environ["CUDA_VISIBLE_DEVICES"] = '2'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2021,type=int,help='The random seed')
    argparser.add_argument('--mode',default='vx',type=str,help='The choice of v/x/vx/xx')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=True,type=bool,help='The use of GPU')
    # About data setting ~~~~
    argparser.add_argument('--num',default=10000,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--num_reps',default=10,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--ate',default=0,type=float,help='The ate of constant')
    argparser.add_argument('--mV',default=2,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mX',default=4,type=int,help='The dim of Confounding variables X')
    argparser.add_argument('--mU',default=4,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--mXs',default=2,type=int,help='The dim of Noise variables D')
    argparser.add_argument('--storage_path',default='../Data/',type=str,help='The dir of data storage')
    # About Regression_t
    argparser.add_argument('--regt_batch_size',default=500,type=int,help='The size of one batch')
    argparser.add_argument('--regt_lr',default=0.05,type=float,help='The learning rate')
    argparser.add_argument('--regt_num_epoch',default=3,type=int,help='The num of total epoch')
    args = argparser.parse_args(args=[])
    return args

args = get_args()

if args.use_gpu:
    device = torch.device('cuda' if torch.cuda.is_available() and args.use_gpu else "cpu")

In [5]:
# run vx
for data in Datasets:
    which_benchmark = data.which_benchmark
    which_dataset = data.which_dataset
    args.num_reps = 10
    args.mV = data.mV
    args.mX = data.mX
    args.mU = data.mU
    args.mXs = data.mXs
    args.mode = 'vx'

    resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}/'
    dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
    os.makedirs(os.path.dirname(resultDir), exist_ok=True)
    logfile = f'{resultDir}/log.txt'

    if args.rewrite_log:
        f = open(logfile,'w')
        f.close()

    for exp in range(args.num_reps):
        train_df = pd.read_csv(dataDir + f'{exp}/train.csv')
        print(dataDir)
        val_df = pd.read_csv(dataDir + f'{exp}/val.csv')
        test_df = pd.read_csv(dataDir + f'{exp}/test.csv')
                                                    
        train = CausalDataset(train_df, variables = ['v','u','x','xs','z','p','s','m','t','g','y','f','c'])
        val = CausalDataset(val_df, variables = ['v','u','x','xs','z','p','s','m','t','g','y','f','c'])
        test = CausalDataset(test_df, variables = ['v','u','x','xs','z','p','s','m','t','g','y','f','c'])

        train,val,test = run_Reg(exp, args, dataDir, resultDir, train, val, test, device)

../Data//data/SynOOD_1_0_1_0.05_0.05_1/2_4_4_2/
Exp 0 :this is the 0/3 epochs.
The train accuracy: 76.60 %
The test  accuracy: 75.50 %
Exp 0 :this is the 1/3 epochs.
The train accuracy: 77.50 %
The test  accuracy: 76.50 %
Exp 0 :this is the 2/3 epochs.
The train accuracy: 77.80 %
The test  accuracy: 77.30 %
../Data//data/SynOOD_1_0_1_0.05_0.05_1/2_4_4_2/
Exp 1 :this is the 0/3 epochs.
The train accuracy: 74.70 %
The test  accuracy: 75.60 %
Exp 1 :this is the 1/3 epochs.
The train accuracy: 75.40 %
The test  accuracy: 77.60 %
Exp 1 :this is the 2/3 epochs.
The train accuracy: 76.80 %
The test  accuracy: 78.10 %
../Data//data/SynOOD_1_0_1_0.05_0.05_1/2_4_4_2/
Exp 2 :this is the 0/3 epochs.
The train accuracy: 75.30 %
The test  accuracy: 73.70 %
Exp 2 :this is the 1/3 epochs.
The train accuracy: 78.10 %
The test  accuracy: 74.20 %
Exp 2 :this is the 2/3 epochs.
The train accuracy: 79.60 %
The test  accuracy: 75.50 %
../Data//data/SynOOD_1_0_1_0.05_0.05_1/2_4_4_2/
Exp 3 :this is the 0/3 ep

In [25]:
data = Datasets[0]
which_benchmark = data.which_benchmark
which_dataset = data.which_dataset
args.num_reps = 10
args.mV = data.mV
args.mX = data.mX
args.mU = data.mU
args.mD = data.mXs
args.mode = 'vx'

resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}/'
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
os.makedirs(os.path.dirname(resultDir), exist_ok=True)
logfile = f'{resultDir}/log.txt'

train_df = pd.read_csv(dataDir + f'{0}/train.csv')
train = CausalDataset(train_df, variables = ['v','u','x','xs','z','p','s','m','t','g','y','f','c'])


In [2]:
import sys

import os
import argparse
import pandas as pd
import numpy as np
import torch
sys.path.append(r"../")
sys.path.append('/home/wyliu/code/CB-IV')
from utils import log, CausalDataset
from module.SynCBIV import run as run_SynCBIV

os.environ["CUDA_VISIBLE_DEVICES"] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2021,type=int,help='The random seed')
    argparser.add_argument('--mode',default='vx',type=str,help='The choice of v/x/vx/xx')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=False,type=bool,help='The use of GPU')
    # About data setting ~~~~
    argparser.add_argument('--num',default=10000,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--num_reps',default=10,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--ate',default=0,type=float,help='The ate of constant')
    argparser.add_argument('--sc',default=1,type=float,help='The sc')
    argparser.add_argument('--sh',default=0,type=float,help='The sh')
    argparser.add_argument('--one',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--depX',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--depU',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--VX',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mV',default=2,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mX',default=4,type=int,help='The dim of Confounding variables X')
    argparser.add_argument('--mU',default=4,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--mXs',default=2,type=int,help='The dim of Noise variables D')
    argparser.add_argument('--storage_path',default='../Data/',type=str,help='The dir of data storage')
    # Syn
    argparser.add_argument('--syn_alpha',default=0.01,type=float,help='')
    argparser.add_argument('--syn_lambda',default=0.0001,type=float,help='')
    argparser.add_argument('--syn_twoStage',default=True,type=bool,help='')
    # About Debug or Show
    argparser.add_argument('--verbose',default=1,type=int,help='The level of verbose')
    argparser.add_argument('--epoch_show',default=5,type=int,help='The epochs of show time')
    args = argparser.parse_args(args=[])
    return args

args = get_args()

In [5]:
# set path
which_benchmark = 'SynOOD_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one, args.depX, args.depU,args.VX])
which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU, args.mXs])
resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}/'
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
os.makedirs(os.path.dirname(resultDir), exist_ok=True)
logfile = f'{resultDir}/log.txt'
exp = 0

In [6]:
dataDir

'../Data//data/SynOOD_1_0_1_0.05_0.05_1/2_4_4_2/'

In [7]:
''' bias rate '''
br = [-3.0, -2.5, -2.0, -1.5, -1.3, 1.3, 1.5, 2.0, 2.5, 3.0, 0.0]
brdc = {-3.0: 'n30', -2.5:'n25', -2.0:'n20', -1.5:'n15', -1.3:'n13', 1.3:'p13', 1.5:'p15', 2.0:'p20', 2.5:'p25', 3.0:'p30', 0.0:'0'}

In [8]:
from utils import * 
for perm in range(args.num_reps):
    for r in br:
        print(r)
        train_df = pd.read_csv(dataDir + f'{perm}/{args.mode}/train.csv')
        val_df = pd.read_csv(dataDir + f'{perm}/{args.mode}/val.csv')
        test_df = pd.read_csv(dataDir + f'{perm}/{args.mode}/test.csv')

        train_df_ood = correlation_sample(train_df, r, 1000, args.mXs)
        val_df_ood = correlation_sample(val_df, r, 1000, args.mXs)
        test_df_ood = correlation_sample(test_df, r, 1000, args.mXs)

        path = dataDir + '/{}/{}/'.format(perm, args.mode)
        os.makedirs(os.path.dirname(path + f'ood_{brdc[r]}/'), exist_ok=True)

        train_df_ood.to_csv(path + f'ood_{brdc[r]}/train.csv', index=False)
        val_df_ood.to_csv(path + f'ood_{brdc[r]}/val.csv', index=False)
        test_df_ood.to_csv(path + f'ood_{brdc[r]}/test.csv', index=False)

-3.0
-2.5
-2.0
-1.5
-1.3
1.3
1.5
2.0
2.5
3.0
0.0
-3.0
-2.5
-2.0
-1.5
-1.3
1.3
1.5
2.0
2.5
3.0
0.0
-3.0
-2.5
-2.0
-1.5
-1.3
1.3
1.5
2.0
2.5
3.0
0.0
-3.0
-2.5
-2.0
-1.5
-1.3
1.3
1.5
2.0
2.5
3.0
0.0
-3.0
-2.5
-2.0
-1.5
-1.3
1.3
1.5
2.0
2.5
3.0
0.0
-3.0
-2.5
-2.0
-1.5
-1.3
1.3
1.5
2.0
2.5
3.0
0.0
-3.0
-2.5
-2.0
-1.5
-1.3
1.3
1.5
2.0
2.5
3.0
0.0
-3.0
-2.5
-2.0
-1.5
-1.3
1.3
1.5
2.0
2.5
3.0
0.0
-3.0
-2.5
-2.0
-1.5
-1.3
1.3
1.5
2.0
2.5
3.0
0.0
-3.0
-2.5
-2.0
-1.5
-1.3
1.3
1.5
2.0
2.5
3.0
0.0


In [40]:
from utils import * 
r = 1.0
# for perm in range(args.num_reps):
train_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/train.csv')
val_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/val.csv')
test_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/test.csv')

In [41]:
data = train_df
n = 1000
dim_xs = 2

In [42]:
nall = data.shape[0]
prob = np.ones(nall)

ite = data['m1']-data['m0']

In [44]:
idx = np.random.choice(range(nall), n, p=prob, replace=False)

In [29]:
idv = 0
d = np.abs(data[f'xs{idv}'] - np.sign(r) * ite) 

In [39]:
np.sum(prob)

1.0000000000000004

In [35]:
prob

array([1., 1., 1., ..., 1., 1., 1.])